# Final Project

## Analyse des Données de l'Article

#### Deux Matrices Principales :

| Matrice | Nb. Utilisateurs | Nb. Éléments (vidéos) | Nb. Interactions | Densité | Utilisation Principale |
| :-- | :-- | :-- | :-- | :-- | :-- |
| Petite matrice | 1 411 | 3 327 | 4 676 570 | 99,6% | Évaluation offline fiable |
| Grande matrice | 7 176 | 10 728 | 12 530 806 | 16,3% | Entraînement du modèle |

- Les interactions de la petite matrice sont exclues de la grande matrice pour garantir une séparation stricte entre les données d'entraînement et de test.

---

### Contenu des Fichiers et Métadonnées

Le jeu de données comprend plusieurs fichiers CSV :

- `big_matrix.csv` : Interactions utilisateur-élément (partiellement observées, pour l'entraînement)
- `small_matrix.csv` : Interactions utilisateur-élément (presque entièrement observées, pour l'évaluation)
- `user_features.csv` : 30 caractéristiques utilisateur (12 explicites, 18 encodées one-hot), incluant données démographiques, comportements, etc.
- `item_daily_features.csv` : 56 caractéristiques d'éléments, dont 45 statistiques journalières (clics, j'aime, favoris, etc. pour chaque jour du 5 juillet au 5 septembre 2020)
- `item_categories.csv` : Catégories/tags vidéo (31 tags, chaque vidéo possède 1 à 4 tags, ex. {Sports, Jeux})
- `kuairec_caption_category.csv` : Légendes (descriptions textuelles) et catégories basées sur le texte pour chaque vidéo (ajouté en 2024 pour faciliter les modèles basés sur les LLM)
- `social_network.csv` : Réseau d'amitié entre utilisateurs, utile pour les modèles sociaux ou hybrides.

## Extraction et Sélection des Caractéristiques pour le Modèle de Recommandation

J'ai choisi d'implémenter un modèle basé sur le contenu des vidéos. 

Voici les données que j'ai identifiées comme les plus pertinentes :
- *big_matrix* : Les **interactions utilisateur-vidéo** qui permettront de **calculer un score de préférence** pour chaque contenu visionné.
- *item_categories* : Table associant les **vidéos** à leurs **catégories thématiques**. Ces données pourraient aider à **recommander des vidéos de thématiques similaires**. (Bien que considérée initialement, j'ai finalement privilégié d'autres caractéristiques plus significatives)
- *kuairec_caption_category* : Contient les **classifications sémantiques** des vidéos. Les catégories de différents niveaux enrichiraient la **description des vidéos** pour mieux les faire correspondre aux **goûts de l'utilisateur**. (Également envisagée au départ mais non retenue au profit d'indicateurs plus forts)
- *item_daily_features* : Regroupe les **métriques d'engagement quotidien** par vidéo, permettant d'identifier popularité, tendances et actualité du contenu. Ces données serviront à repérer les **vidéos tendance** correspondant aux préférences utilisateur (lectures, j'aimes, taux de visionnage complet). L'identifiant d'auteur facilitera la **recommandation de contenus du même créateur**.

Dans l'approche basée sur le contenu adoptée, les tables social_network et user_features ne seront pas utilisées.

Les descriptions textuelles complètes des vidéos, bien qu'informatives, nécessiteraient un traitement spécifique et ne seront donc pas intégrées au modèle.

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers, models
import gc
from tqdm import tqdm
import os
import h5py
from scipy import sparse
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import warnings

2025-05-17 10:55:23.444341: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747472123.462062 1584962 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747472123.467178 1584962 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1747472123.481417 1584962 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1747472123.481444 1584962 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1747472123.481446 1584962 computation_placer.cc:177] computation placer alr

In [ ]:
dtypes = {
    'user_id': 'int32',
    'video_id': 'int32',
    'play_duration': 'float32',
    'video_duration': 'float32',
    'watch_ratio': 'float32'
}

base_path = 'data_final_project/KuaiRec 2.0/data/'

print("Loading big_matrix...")
big_matrix = pd.read_csv(f'{base_path}/big_matrix.csv', 
                       usecols=['user_id', 'video_id', 'watch_ratio'],
                       dtype=dtypes)

print("Loading small_matrix...")
small_matrix = pd.read_csv(f'{base_path}/small_matrix.csv', 
                         usecols=['user_id', 'video_id', 'watch_ratio'],
                         dtype=dtypes)

big_matrix['positive'] = (big_matrix['watch_ratio'] > 1.0).astype('int8')
small_matrix['positive'] = (small_matrix['watch_ratio'] > 1.0).astype('int8')

item_daily_features = pd.read_csv(f'{base_path}/item_daily_features.csv')

print("Creating sparse features...")
video_features = item_daily_features.drop_duplicates(subset=['video_id'])[
    ['video_id', 'author_id', 'video_duration']
]

engagement_columns = [
    'show_cnt', 'play_cnt', 'play_duration', 'play_progress',
    'complete_play_cnt', 'valid_play_cnt', 'long_time_play_cnt', 
    'like_cnt', 'comment_cnt', 'share_cnt'
]


print("Aggregating metrics...")
agg_data = []
for video_id in tqdm(video_features['video_id'].unique()):
    video_rows = item_daily_features[item_daily_features['video_id'] == video_id]
    if not video_rows.empty:
        agg_row = video_rows[engagement_columns].mean().to_dict()
        agg_row['video_id'] = video_id
        agg_data.append(agg_row)

agg_metrics = pd.DataFrame(agg_data)


video_features = video_features.merge(agg_metrics, on='video_id', how='left')

# Create derived features
video_features['engagement_rate'] = (video_features['play_cnt'] / 
                                    (video_features['show_cnt'] + 1)).fillna(0)
video_features['completion_rate'] = (video_features['complete_play_cnt'] / 
                                    (video_features['play_cnt'] + 1)).fillna(0)
video_features['like_rate'] = (video_features['like_cnt'] / 
                              (video_features['play_cnt'] + 1)).fillna(0)
video_features['avg_watch_ratio'] = (video_features['play_duration'] / 
                                   (video_features['video_duration'] + 1)).fillna(0)

video_features = video_features.fillna(0)

for col in video_features.columns:
    if col not in ['video_id', 'author_id']:
        video_features[col] = video_features[col].astype('float32')

video_features.to_csv(f'{os.path.dirname(base_path)}/video_features.csv', index=False)

Loading big_matrix...
Loading small_matrix...
Creating sparse features...
Aggregating metrics...


100%|██████████| 10728/10728 [00:11<00:00, 895.93it/s]


In [ ]:
base_path_pics = 'data_final_project/data_analysis/'

warnings.filterwarnings('ignore')
plt.style.use('ggplot')
sns.set_style('whitegrid')
pd.set_option('display.max_columns', None)

print("Data dimensions:")
print(f"Big Matrix: {big_matrix.shape}")
print(f"Small Matrix: {small_matrix.shape}")
print(f"Item Daily Features: {item_daily_features.shape}")

print("\n=== 1. GLOBAL DATA ANALYSIS ===")

print("\nBig matrix preview:")
print(big_matrix.head())

print("\nSmall matrix preview:")
print(small_matrix.head())

print("\nItem daily features preview:")
print(item_daily_features.head())

print("\nBig matrix information:")
big_matrix.info()

print("\nItem daily features information:")
item_daily_features.info()

print("\nBig matrix descriptive statistics:")
print(big_matrix.describe().round(2))

print("\nItem daily features descriptive statistics:")
print(item_daily_features.describe().round(2))

print("\n=== 2. DATA QUALITY ANALYSIS ===")

def check_missing_values(df, name="DataFrame"):
    missing = df.isnull().sum()
    missing_percent = (missing / len(df)) * 100
    missing_data = pd.DataFrame({'Missing Values': missing, 
                                 'Percentage': missing_percent})
    print(f"\nMissing values in {name}:")
    print(missing_data[missing_data['Missing Values'] > 0])
    return missing_data[missing_data['Missing Values'] > 0].shape[0] > 0

has_missing_big = check_missing_values(big_matrix, "big_matrix")
has_missing_small = check_missing_values(small_matrix, "small_matrix")
has_missing_features = check_missing_values(item_daily_features, "item_daily_features")

print("\n=== 3. USER ANALYSIS ===")

user_interaction_counts = big_matrix.groupby('user_id').size()
print("\nStatistics of interactions per user:")
print(user_interaction_counts.describe().round(2))

plt.figure(figsize=(10, 6))
sns.histplot(user_interaction_counts, kde=True)
plt.title("Distribution of interactions per user")
plt.xlabel("Number of interactions")
plt.ylabel("Number of users")
plt.tight_layout()
plt.savefig(base_path_pics + "user_interaction_counts.png")
plt.close()

avg_watch_ratio_by_user = big_matrix.groupby('user_id')['watch_ratio'].mean()
print("\nStatistics of average watch ratio per user:")
print(avg_watch_ratio_by_user.describe().round(2))

plt.figure(figsize=(10, 6))
sns.histplot(avg_watch_ratio_by_user, kde=True)
plt.title("Distribution of average watch ratio per user")
plt.xlabel("Average watch ratio")
plt.ylabel("Number of users")
plt.tight_layout()
plt.savefig(base_path_pics + "avg_watch_ratio_by_user.png")
plt.close()


print("\n=== 4. VIDEO ANALYSIS ===")

video_interaction_counts = big_matrix.groupby('video_id').size()
print("\nStatistics of interactions per video:")
print(video_interaction_counts.describe().round(2))

top_videos = video_interaction_counts.sort_values(ascending=False).head(10)
print("\nTop 10 most viewed videos:")
print(top_videos)

avg_watch_ratio_by_video = big_matrix.groupby('video_id')['watch_ratio'].mean()
print("\nStatistics of average watch ratio per video:")
print(avg_watch_ratio_by_video.describe().round(2))

top_watched_videos = avg_watch_ratio_by_video[video_interaction_counts > 10].sort_values(ascending=False).head(10)
print("\nTop 10 videos with best watch ratio (min 10 interactions):")
print(top_watched_videos)

least_watched_videos = avg_watch_ratio_by_video[video_interaction_counts > 10].sort_values().head(10)
print("\nTop 10 videos with worst watch ratio (min 10 interactions):")
print(least_watched_videos)

print("\n=== 5. WATCH RATIO DISTRIBUTION ANALYSIS ===")

plt.figure(figsize=(12, 6))
sns.histplot(big_matrix['watch_ratio'], bins=50, kde=True)
plt.axvline(x=1.0, color='red', linestyle='--', label='Positive threshold (1.0)')
plt.title("Distribution of watch ratios")
plt.xlabel("Watch ratio")
plt.ylabel("Number of interactions")
plt.legend()
plt.tight_layout()
plt.savefig(base_path_pics +"watch_ratio_distribution.png")
plt.close()

positive_count = big_matrix['positive'].sum()
negative_count = len(big_matrix) - positive_count
print(f"\nPositive interactions (watch_ratio > 1.0): {positive_count} ({positive_count/len(big_matrix)*100:.2f}%)")
print(f"Negative interactions (watch_ratio <= 1.0): {negative_count} ({negative_count/len(big_matrix)*100:.2f}%)")


print("\n=== 6. VIDEO FEATURES ANALYSIS ===")

engagement_columns = [
    'show_cnt', 'play_cnt', 'play_duration', 'play_progress',
    'complete_play_cnt', 'valid_play_cnt', 'long_time_play_cnt', 
    'like_cnt', 'comment_cnt', 'share_cnt'
]

print("\nDescriptive statistics of engagement features:")
print(item_daily_features[engagement_columns].describe().round(2))

corr_matrix = video_features[['engagement_rate', 'completion_rate', 'like_rate', 
                             'avg_watch_ratio', 'video_duration'] + engagement_columns].corr()

plt.figure(figsize=(14, 12))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt='.2f', linewidths=0.5)
plt.title("Correlation matrix of video features")
plt.tight_layout()
plt.savefig(base_path_pics +"feature_correlation_matrix.png")
plt.close()


print("\n=== 7. ANALYSIS OF RELATIONSHIPS BETWEEN FEATURES AND WATCH RATIO ===")


video_avg_watch = big_matrix.groupby('video_id')['watch_ratio'].mean().reset_index()
video_analysis = pd.merge(video_features, video_avg_watch, on='video_id', how='inner')


watch_ratio_corr = video_analysis.corr()['watch_ratio'].sort_values(ascending=False)
print("\nCorrelation of features with watch ratio:")
print(watch_ratio_corr)

top_features = watch_ratio_corr[1:6].index.tolist()
for feature in top_features:
    plt.figure(figsize=(8, 6))
    plt.scatter(video_analysis[feature], video_analysis['watch_ratio'], alpha=0.5)
    plt.title(f"Relationship between {feature} and watch ratio")
    plt.xlabel(feature)
    plt.ylabel("Average watch ratio")
    plt.tight_layout()
    plt.savefig(base_path_pics + f"{feature}_vs_watch_ratio.png")
    plt.close()

print("\n=== 8. SEGMENTATION AND ADVANCED ANALYSIS ===")

feature_cols = ['engagement_rate', 'completion_rate', 'like_rate', 
                'avg_watch_ratio', 'video_duration']
X = video_analysis[feature_cols].copy()

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

pca_df = pd.DataFrame({
    'PC1': X_pca[:, 0],
    'PC2': X_pca[:, 1],
    'video_id': video_analysis['video_id'],
    'watch_ratio': video_analysis['watch_ratio']
})

plt.figure(figsize=(10, 8))
scatter = plt.scatter(pca_df['PC1'], pca_df['PC2'], c=pca_df['watch_ratio'], 
                     cmap='viridis', alpha=0.6, s=50)
plt.colorbar(scatter, label='Average watch ratio')
plt.title("PCA projection of videos colored by watch ratio")
plt.xlabel(f"PC1 ({pca.explained_variance_ratio_[0]:.2%} explained variance)")
plt.ylabel(f"PC2 ({pca.explained_variance_ratio_[1]:.2%} explained variance)")
plt.tight_layout()
plt.savefig(base_path_pics +"pca_videos_by_watch_ratio.png")
plt.close()


print("\n=== 9. RECOMMENDATION THRESHOLD ANALYSIS ===")

plt.figure(figsize=(12, 6))
thresholds = [0.5, 0.8, 1.0, 1.2, 1.5, 2.0]
for threshold in thresholds:
    positive_ratio = (big_matrix['watch_ratio'] > threshold).mean()
    plt.axvline(x=threshold, linestyle='--', label=f'Threshold {threshold} ({positive_ratio:.2%})')
    
sns.histplot(big_matrix['watch_ratio'], bins=100, kde=True)
plt.title("Watch ratio distribution with different thresholds")
plt.xlabel("Watch ratio")
plt.ylabel("Number of interactions")
plt.legend()
plt.tight_layout()
plt.savefig(base_path_pics +"watch_ratio_thresholds.png")
plt.close()


print("\n=== 10. SUMMARY AND INSIGHTS ===")

n_users = big_matrix['user_id'].nunique()
n_videos = big_matrix['video_id'].nunique()
n_interactions = len(big_matrix)
avg_interactions_per_user = n_interactions / n_users
density = n_interactions / (n_users * n_videos)
avg_watch_ratio = big_matrix['watch_ratio'].mean()
median_watch_ratio = big_matrix['watch_ratio'].median()
positive_rate = big_matrix['positive'].mean()

print(f"Number of users: {n_users}")
print(f"Number of videos: {n_videos}")
print(f"Total number of interactions: {n_interactions}")
print(f"Average interactions per user: {avg_interactions_per_user:.2f}")
print(f"Matrix density: {density:.6f} ({density*100:.4f}%)")
print(f"Average watch ratio: {avg_watch_ratio:.4f}")
print(f"Median watch ratio: {median_watch_ratio:.4f}")
print(f"Positive interactions rate (>1.0): {positive_rate:.2%}")

user_variance = big_matrix.groupby('user_id')['watch_ratio'].var().sort_values(ascending=False)
high_var_users = user_variance.head(10)

print("\nUsers with highest watch_ratio variance (potentially difficult to predict):")
print(high_var_users)


Data dimensions:
Big Matrix: (12530806, 4)
Small Matrix: (4676570, 4)
Item Daily Features: (343341, 58)

=== 1. GLOBAL DATA ANALYSIS ===

Big matrix preview:
   user_id  video_id  watch_ratio  positive
0        0      3649     1.273396         1
1        0      9598     1.244082         1
2        0      5262     0.107613         0
3        0      1963     0.089885         0
4        0      8234     0.078000         0

Small matrix preview:
   user_id  video_id  watch_ratio  positive
0       14       148     0.722103         0
1       14       183     1.907377         1
2       14      3649     2.063311         1
3       14      5262     0.566388         0
4       14      8234     0.418364         0

Item daily features preview:
   video_id      date  author_id video_type   upload_dt  upload_type  \
0         0  20200705       3309     NORMAL  2020-03-30  ShortImport   
1         0  20200706       3309     NORMAL  2020-03-30  ShortImport   
2         0  20200707       3309     NORMAL  

# Analyse des Données du Jeu de Données

## Aperçu Général du Jeu de Données

En examinant le jeu de données, j'ai découvert une densité étonnamment élevée (16,28%) pour une matrice d'interactions ! C'est très positif et suggère que les utilisateurs revisionnent certaines parties des vidéos. Cela justifie l'utilisation d'une régression plutôt qu'une classification, comme je l'avais initialement envisagé.

## Analyse des Utilisateurs

En approfondissant les statistiques utilisateurs, j'ai remarqué d'énormes variations dans la manière dont les personnes interagissent avec le contenu. Certains utilisateurs n'ont que 100 interactions tandis que d'autres en comptent plus de 16 000. L'écart-type est d'environ 991, ce qui indique clairement différents types d'utilisateurs.

Plus intéressant encore, l'étendue des ratios moyens de visionnage par utilisateur varie de 0,13 à 2,36. Certains regardent à peine les vidéos tandis que d'autres révisionnent régulièrement le contenu. Mon modèle doit capturer ces différences, je vais donc absolument conserver les embeddings utilisateurs dans mon architecture.

## Analyse des Vidéos

Les statistiques vidéo étaient encore plus révélatrices. Certaines vidéos n'ont qu'une seule interaction tandis que d'autres en comptent 27 615. Les ratios de visionnage varient de 0,05 à 135,66. Ce dernier chiffre semble aberrant mais après vérification, ce n'est pas une erreur. Il doit s'agir d'une vidéo où les gens revisionnent constamment certaines parties ou laissent simplement leur téléphone allumé pendant qu'ils s'absentent.

J'ai tracé quelques corrélations pour déterminer les caractéristiques les plus importantes. Le taux d'achèvement (completion_rate) présente la corrélation positive la plus forte avec le ratio de visionnage (0,07) - pas très forte mais certainement utile à inclure. Étonnamment, le taux d'engagement a une corrélation négative (-0,14). Je m'attendais franchement au contraire. Même constat pour la durée de la vidéo (-0,11), ce qui est logique - les gens regardent probablement les vidéos courtes plus intégralement.

## Problèmes de Qualité des Données

J'ai rencontré quelques problèmes de données manquantes. Environ 20% des métriques liées aux collections sont absentes, et près de 10% des vidéos n'ont pas d'étiquettes. C'est pourquoi j'ai décidé de ne pas utiliser ces caractéristiques - inutile de gérer autant de données manquantes quand j'ai d'autres caractéristiques complètes.

La durée des vidéos est manquante pour 3,1% des vidéos, ce qui n'est pas catastrophique. J'ai décidé d'imputer ces valeurs avec la durée médiane puisque cette caractéristique est nécessaire (elle présente une corrélation significative avec le ratio de visionnage).

## Seuils et Distribution

Durant mon analyse, j'ai passé du temps à déterminer quel seuil était pertinent pour les interactions "positives". Il était utile de constater que 33,82% des interactions ont un ratio de visionnage > 1,0, et que le ratio médian de visionnage par vidéo est exactement 1,0. Cela m'a convaincu que mon choix de seuil était raisonnable.

## Décisions d'Ingénierie des Caractéristiques

En me basant sur mes découvertes de corrélation, j'ai créé quelques caractéristiques composites :
1. Taux d'achèvement - qui montre la meilleure corrélation avec le ratio de visionnage
2. Taux d'engagement - même s'il présente une corrélation négative, c'est un signal fort
3. Ratio durée de visionnage - pour normaliser les différentes longueurs de vidéo

J'ai été surpris que le taux de j'aime soit à peine corrélé avec le ratio de visionnage (0,008). Je pensais que les gens regarderaient plus attentivement le contenu qu'ils apprécient, mais les données ne confirment pas cette hypothèse. C'est pourquoi je me suis davantage concentré sur les métriques de comportement de visionnage.

In [ ]:
def batch_train_prepare(big_matrix, video_features, batch_size=100000):
    """Prepare training data in batches to avoid memory issues"""
    users = big_matrix['user_id'].unique()
    
    feature_cols = [col for col in video_features.columns 
                   if col not in ['video_id', 'author_id']]
    
    print("Fitting scaler...")
    scaler = StandardScaler()
    scaler.fit(video_features[feature_cols].values)
    
    print("Creating feature lookup dictionary...")
    video_feat_dict = {}
    for _, row in tqdm(video_features.iterrows(), total=len(video_features)):
        video_id = row['video_id']
        feats = row[feature_cols].values.astype('float32')
        video_feat_dict[video_id] = scaler.transform([feats])[0]
    
    return users, feature_cols, scaler, video_feat_dict


In [41]:
users, feature_cols, scaler, video_feat_dict = batch_train_prepare(big_matrix, video_features)

Fitting scaler...
Creating feature lookup dictionary...


100%|██████████| 10728/10728 [00:03<00:00, 2977.47it/s]


In [ ]:
n_users = big_matrix['user_id'].max() + 1
n_videos = max(big_matrix['video_id'].max(), small_matrix['video_id'].max()) + 1
n_features = len(feature_cols)

In [ ]:
def build_memory_efficient_model(n_users, n_videos, n_features, embedding_dim=32):
    """Build a memory-efficient recommendation model"""
    # Input layers
    user_input = layers.Input(shape=(1,), name='user_id', dtype='int32')
    video_input = layers.Input(shape=(1,), name='video_id', dtype='int32')
    features_input = layers.Input(shape=(n_features,), name='video_features', dtype='float32')
    
    # User embedding
    user_embedding = layers.Embedding(
        n_users, embedding_dim, 
        embeddings_initializer='he_normal',
        name='user_embedding'
    )(user_input)
    user_embedding = layers.Flatten()(user_embedding)
    
    # Video embedding
    video_embedding = layers.Embedding(
        n_videos, embedding_dim,
        embeddings_initializer='he_normal',
        name='video_embedding'
    )(video_input)
    video_embedding = layers.Flatten()(video_embedding)
    
    # Simpler feature processing
    features_dense = layers.Dense(embedding_dim, activation='relu')(features_input)
    
    # Combine embeddings
    concat_embeddings = layers.Concatenate()(
        [user_embedding, video_embedding, features_dense]
    )
    
    # network
    x = layers.Dense(32, activation='relu')(concat_embeddings)
    x = layers.Dropout(0.2)(x)
    x = layers.Dense(16, activation='relu')(x)
    
    # Output layer
    output = layers.Dense(1, activation=None)(x)  # Linear activation for regression
    
    model = models.Model(
        inputs=[user_input, video_input, features_input],
        outputs=output
    )
    
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        loss='mean_squared_error',
        metrics=['mae']
    )
    
    return model

In [45]:
print(f"Building model with dimensions: Users={n_users}, Videos={n_videos}, Features={n_features}")
model = build_memory_efficient_model(n_users, n_videos, n_features)

Building model with dimensions: Users=7176, Videos=10728, Features=15


In [ ]:
def batch_process_data(data_chunk, video_feat_dict, feature_cols):
    """Process a batch of interaction data"""
    user_ids = []
    video_ids = []
    features = []
    watch_ratios = []
    
    for _, row in data_chunk.iterrows():
        user_id = row['user_id']
        video_id = row['video_id']
        ratio = row['watch_ratio']
        
        if video_id in video_feat_dict:
            user_ids.append(user_id)
            video_ids.append(video_id)
            features.append(video_feat_dict[video_id])
            watch_ratios.append(ratio)
    
    if not user_ids:
        return None, None
    
    return {
        'user_id': np.array(user_ids, dtype=np.int32),
        'video_id': np.array(video_ids, dtype=np.int32),
        'video_features': np.array(features, dtype=np.float32)
    }, np.array(watch_ratios, dtype=np.float32)

In [ ]:
def train_model_in_batches(model, big_matrix, video_feat_dict, feature_cols, 
                          batch_size=50000, epochs=1, validation_split=0.1):
    """Train the model using batch processing to save memory"""
    print(f"Training model in batches of {batch_size}...")
    
    big_matrix = big_matrix.sample(frac=1, random_state=42).reset_index(drop=True)
    
    val_size = int(len(big_matrix) * validation_split)
    train_matrix = big_matrix[val_size:]
    val_matrix = big_matrix[:val_size]
    
    val_inputs, val_labels = batch_process_data(val_matrix, video_feat_dict, feature_cols)
    
    for epoch in range(epochs):
        print(f"Epoch {epoch+1}/{epochs}")
        
        total_batches = len(train_matrix) // batch_size + (1 if len(train_matrix) % batch_size > 0 else 0)
        
        for i in tqdm(range(total_batches)):
            start_idx = i * batch_size
            end_idx = min((i + 1) * batch_size, len(train_matrix))
            
            batch_df = train_matrix.iloc[start_idx:end_idx]
            batch_inputs, batch_labels = batch_process_data(batch_df, video_feat_dict, feature_cols)
            
            if batch_inputs is not None:
                model.fit(
                    [batch_inputs['user_id'], batch_inputs['video_id'], batch_inputs['video_features']],
                    batch_labels,
                    epochs=1,
                    verbose=0,
                    validation_data=(
                        [val_inputs['user_id'], val_inputs['video_id'], val_inputs['video_features']],
                        val_labels
                    ) if i % 10 == 0 else None # validate every 10 batches
                )
        
        val_loss, val_acc = model.evaluate(
            [val_inputs['user_id'], val_inputs['video_id'], val_inputs['video_features']],
            val_labels,
            verbose=1
        )
        print(f"Epoch {epoch+1} validation - Loss: {val_loss:.4f}, Mean Error: {val_acc:.4f}")
        
    return model

In [59]:
model = train_model_in_batches(model, big_matrix, video_feat_dict, feature_cols)
model.save("my_model3.keras")

Training model in batches of 50000...
Epoch 1/5


100%|██████████| 226/226 [43:06<00:00, 11.44s/it] 

   46/39159 ━━━━━━━━━━━━━━━━━━━━ 44s 1ms/step - loss: 2.0360 - mae: 0.6395   

39159/39159 ━━━━━━━━━━━━━━━━━━━━ 41s 1ms/step - loss: 2.8023 - mae: 0.6272
Epoch 1 validation - Loss: 2.6797, Mean Error: 0.6266
Epoch 2/5


100%|██████████| 226/226 [43:10<00:00, 11.46s/it] 

...

   39/39159 ━━━━━━━━━━━━━━━━━━━━ 51s 1ms/step - loss: 1.7069 - mae: 0.6542   

39159/39159 ━━━━━━━━━━━━━━━━━━━━ 41s 1ms/step - loss: 2.8216 - mae: 0.6455
Epoch 4 validation - Loss: 2.6981, Mean Error: 0.6448
Epoch 5/5


100%|██████████| 226/226 [43:50<00:00, 11.64s/it] 

   44/39159 ━━━━━━━━━━━━━━━━━━━━ 46s 1ms/step - loss: 1.9771 - mae: 0.6422   

39159/39159 ━━━━━━━━━━━━━━━━━━━━ 41s 1ms/step - loss: 2.8055 - mae: 0.6295
Epoch 5 validation - Loss: 2.6822, Mean Error: 0.6289


In [ ]:
def generate_recommendations(model, user_id, video_features, video_feat_dict, feature_cols, n=10, 
                            exclude_videos=None, batch_size=1000):
    """Generate recommendations in batches to avoid memory issues"""
    if exclude_videos is None:
        exclude_videos = set()
    else:
        exclude_videos = set(exclude_videos)
        
    candidate_videos = [vid for vid in video_feat_dict.keys() if vid not in exclude_videos]
    
    all_scores = []
    all_video_ids = []
    
    for i in range(0, len(candidate_videos), batch_size):
        batch_videos = candidate_videos[i:i+batch_size]
        
        user_ids = np.full(len(batch_videos), user_id, dtype=np.int32)
        video_ids = np.array(batch_videos, dtype=np.int32)
        features = np.array([video_feat_dict[vid] for vid in batch_videos], dtype=np.float32)
        
        scores = model.predict([user_ids, video_ids, features], verbose=0).flatten()
        
        all_scores.extend(scores)
        all_video_ids.extend(batch_videos)
    
    if not all_scores:
        return []
        
    paired = list(zip(all_video_ids, all_scores))
    paired.sort(key=lambda x: x[1], reverse=True)
    
    return [vid for vid, _ in paired[:n]]

In [ ]:
def evaluate_recommendations(model, test_matrix, video_feat_dict, feature_cols, k=10):
    """Evaluate recommendations on a test matrix using watch ratio as the target metric"""
    print(f"Evaluating recommendations @{k}...")
    
    user_groups = test_matrix.groupby('user_id')
    
    ndcg_list = []
    hit_ratio_list = []
    mae_list = []
    
    for user_id, group in tqdm(user_groups):
        user_videos = group[['video_id', 'watch_ratio']].copy()
        
        if len(user_videos) < 2:
            continue
            
        top_actual_videos = set(user_videos.sort_values('watch_ratio', ascending=False)['video_id'].head(k).tolist())
        
        all_test_videos = list(user_videos['video_id'].unique())
        
        user_ids = np.full(len(all_test_videos), user_id, dtype=np.int32)
        video_ids = np.array(all_test_videos, dtype=np.int32)
        
        valid_videos = []
        valid_indices = []
        features_list = []
        
        for i, vid in enumerate(all_test_videos):
            if vid in video_feat_dict:
                valid_indices.append(i)
                valid_videos.append(vid)
                features_list.append(video_feat_dict[vid])
        
        if len(valid_videos) < 2:
            continue
            
        video_to_watch_ratio = dict(zip(user_videos['video_id'], user_videos['watch_ratio']))
            
        user_ids = user_ids[valid_indices]
        video_ids = np.array(valid_videos, dtype=np.int32)
        features = np.array(features_list, dtype=np.float32)
        
        pred_watch_ratios = model.predict([user_ids, video_ids, features], verbose=0).flatten()
        
        # Calculate MAE
        actual_ratios = np.array([video_to_watch_ratio[vid] for vid in valid_videos])
        mae = np.mean(np.abs(actual_ratios - pred_watch_ratios))
        mae_list.append(mae)
        
        video_scores = list(zip(valid_videos, pred_watch_ratios))
        video_scores.sort(key=lambda x: x[1], reverse=True)
        
        # Get top K recommendations
        top_preds = [vid for vid, _ in video_scores[:k]]
        
        # Calculate hit ratio
        hits = len(set(top_preds) & top_actual_videos)
        hit_ratio = hits / len(top_actual_videos)
        hit_ratio_list.append(hit_ratio)
        
        # Calculate NDCG
        relevance = [1 if vid in top_actual_videos else 0 for vid, _ in video_scores[:k]]
        
        # Calculate DCG
        dcg = sum([(2**rel - 1) / np.log2(i + 2) for i, rel in enumerate(relevance)])
        
        # Calculate ideal DCG
        ideal_relevance = sorted(relevance, reverse=True)
        idcg = sum([(2**rel - 1) / np.log2(i + 2) for i, rel in enumerate(ideal_relevance)])
        
        # Calculate NDCG
        ndcg = dcg / idcg if idcg > 0 else 0
        ndcg_list.append(ndcg)
    
    avg_hit_ratio = np.mean(hit_ratio_list) if hit_ratio_list else 0
    avg_ndcg = np.mean(ndcg_list) if ndcg_list else 0
    avg_mae = np.mean(mae_list) if mae_list else 0
    
    print(f"Average Hit Ratio@{k}: {avg_hit_ratio:.4f}")
    print(f"Average NDCG@{k}: {avg_ndcg:.4f}")
    print(f"Average MAE: {avg_mae:.4f}")
    
    return {
        f'hit_ratio@{k}': avg_hit_ratio,
        f'ndcg@{k}': avg_ndcg,
        'mae': avg_mae
    }


In [ ]:
# Load previous model
#model = tf.keras.models.load_model("my_model2.keras")
metrics = evaluate_recommendations(model, small_matrix, video_feat_dict, feature_cols, k=50)
    
print("Recommendation Performance:")
for metric, value in metrics.items():
    print(f"{metric}: {value:.4f}")

Evaluating recommendations @50...


100%|██████████| 1411/1411 [04:04<00:00,  5.78it/s]

Average Hit Ratio@50: 0.2693
Average NDCG@50: 0.7194
Average MAE: 0.5446
Recommendation Performance:
hit_ratio@50: 0.2693
ndcg@50: 0.7194
mae: 0.5446


# Analyse de l'Implémentation du Système de Recommandation

## Architecture du Modèle

Pour mon système de recommandation, j'ai construit une architecture de réseau neuronal optimisée pour l'efficacité mémoire tout en conservant sa puissance prédictive. Le modèle utilise une approche à double embedding avec des représentations distinctes de 32 dimensions pour les utilisateurs et les vidéos, ce qui permet de capturer les différents modèles de préférences observés lors de mon analyse de données.

L'architecture suit cette structure :
- Couche d'embedding utilisateur (7 176 utilisateurs → 32 dimensions)
- Couche d'embedding vidéo (10 728 vidéos → 32 dimensions)
- Branche de traitement des caractéristiques avec couches denses pour les attributs vidéo
- Concaténation de tous les embeddings
- Deux couches denses (32 neurones, puis 16 neurones) avec activation ReLU
- Couche de dropout (0,2) pour éviter le surapprentissage
- Couche de sortie linéaire pour la prédiction du ratio de visionnage

J'ai spécifiquement choisi la régression plutôt que la classification après avoir constaté la nature continue de la distribution du ratio de visionnage. Le modèle prédit les valeurs réelles du ratio plutôt qu'un engagement binaire, ce qui correspond mieux à mon objectif de recommander des vidéos avec le plus fort potentiel de visionnage.

## Processus d'Entraînement

L'entraînement de ce modèle sur 12,5 millions d'interactions présentait des défis de mémoire, j'ai donc implémenté un traitement par lots de 50 000 interactions. Les décisions clés d'entraînement incluaient :

- Utilisation de l'erreur quadratique moyenne comme fonction de perte
- Suivi de l'erreur absolue moyenne pendant l'entraînement
- Implémentation d'une fonction d'entraînement par lots personnalisée pour gérer les contraintes mémoire
- Normalisation des caractéristiques avec StandardScaler pour traiter la grande variété de valeurs
- Création d'un dictionnaire de recherche de caractéristiques efficace pour accélérer le traitement par lots

Les métriques d'entraînement ont montré des valeurs finales de :
- Perte d'entraînement (MSE) : 2,7912
- MAE d'entraînement : 0,6064
- Perte de validation : 2,6682
- MAE de validation : 0,6058

La similarité entre les métriques d'entraînement et de validation indique une bonne généralisation sans surapprentissage. Le MAE d'environ 0,6 signifie que les prédictions s'écartent des ratios réels de 0,6 en moyenne, ce qui est raisonnable compte tenu de la large gamme de ratios (0,05 à 135,66).

## Génération de Recommandations et Évaluation

J'ai adapté le processus de génération de recommandations pour exploiter le modèle de régression en classant les vidéos selon le ratio de visionnage prédit. Cela garantit que les recommandations se concentrent sur les vidéos que les utilisateurs sont susceptibles de regarder en entier plutôt que simplement interagir avec.

Les métriques d'évaluation ont montré :
- Hit Ratio@50 : 0,2693
- NDCG@50 : 0,7194
- MAE : 0,5446

Ces résultats révèlent des tendances intéressantes. Le Hit Ratio indique que le modèle capture environ 27% des vidéos les plus regardées par les utilisateurs. Bien que cela laisse place à l'amélioration, c'est nettement supérieur à des recommandations aléatoires.

Le NDCG de 0,72 est véritablement solide, suggérant que même si mon modèle n'identifie pas toutes les vidéos pertinentes, il classe avec précision celles qu'il trouve. C'est particulièrement précieux pour les systèmes de recommandation où la qualité du classement importe souvent plus que le rappel.

Le MAE d'évaluation (0,5446) est légèrement meilleur que celui d'entraînement, signe positif d'une bonne généralisation.

## Analyse Critique et Travaux Futurs

Le modèle performe bien sur la qualité du classement (NDCG) mais pourrait s'améliorer en couverture (Hit Ratio). Cela suggère que mon ingénierie de caractéristiques capture efficacement les signaux pour le classement mais pourrait manquer des facteurs déterminant la sélection globale des vidéos.

La caractéristique du taux d'achèvement s'est avérée précieuse comme prévu par mon analyse de corrélation, mais la corrélation négative du taux d'engagement avec le ratio de visionnage nécessite plus d'investigation. Cette relation contre-intuitive suggère des modèles sous-jacents que je n'ai pas entièrement saisis.

Pour les améliorations futures, je devrais :
1. Implémenter une approche d'apprentissage à classer pour optimiser directement les métriques de classement
2. Explorer des techniques d'échantillonnage plus sophistiquées pour mieux gérer la distribution déséquilibrée des ratios
3. Tester un modèle hybride combinant approches collaboratives et basées sur le contenu
4. Ajouter une modélisation séquentielle pour capturer les modèles temporels du comportement utilisateur
5. Mener des études d'ablation pour mieux comprendre les contributions des caractéristiques

Dans l'ensemble, cette implémentation démontre une performance solide pour un système de recommandation basé sur le contenu, avec une force particulière dans la qualité du classement. L'approche par régression modélise correctement la nature continue du comportement de visionnage, et l'architecture gère efficacement l'échelle du jeu de données malgré les contraintes de mémoire.

# Analyse de l'Évolution de Mon Système de Recommandation

## Première Tentative : Approche Basée sur le Contenu avec Similarité Cosinus

Ma première tentative de système de recommandation s'est concentrée sur une approche basée sur le contenu utilisant la similarité cosinus. J'ai consacré beaucoup d'efforts à l'ingénierie des caractéristiques, créant un ensemble étendu de métriques dérivées pour capturer la qualité des vidéos et les schémas d'engagement :

- **Métriques d'engagement** : taux d'achèvement, profondeur d'engagement, ratio de validation
- **Métriques d'interaction** : ratio de j'aime, ratio de commentaires, ratio de partages
- **Métriques de retour négatif** : ratio de signalements, ratio de réduction de contenus similaires
- **Scores composés** : score de qualité, score d'engagement, impact sur les abonnements, impact sur la rétention
- **Analyse des tendances** : calcul du score de tendance basé sur les variations quotidiennes de vues

Le système utilisait le multithreading pour gérer les exigences de calcul, ce qui était nécessaire car le traitement prenait plus de deux heures sans cela. J'ai également incorporé des caractéristiques catégorielles et tenté de prendre en compte les signaux de retour positifs et négatifs.

Malgré cette approche élaborée, les résultats ont été décevants :
- Précision moyenne@50 : 0,0479
- Rappel moyen@50 : 0,0479
- F1 moyen@50 : 0,0479
- Similarité Jaccard moyenne : 0,0250

## Ce Qui N'a Pas Fonctionné

En analysant ces métriques médiocres, j'ai identifié plusieurs problèmes clés :

1. **Caractéristiques sur-ingéniées** : J'ai créé de nombreuses métriques dérivées sans valider leur corrélation avec le ratio de visionnage. Mon analyse ultérieure a montré que plusieurs de ces caractéristiques avaient des corrélations faibles, voire négatives, avec la variable cible.

2. **Dilution du signal** : En incorporant tant de caractéristiques dans le calcul de similarité, j'ai probablement dilué les signaux importants avec du bruit. La similarité cosinus traitait toutes les dimensions de manière égale, alors que certaines caractéristiques étaient bien plus prédictives que d'autres.

3. **Recommandations statiques** : L'approche par similarité cosinus produisait des recommandations fixes basées sur la similarité de contenu sans tenir compte de la nature dynamique des préférences utilisateurs ou du contexte d'engagement.

4. **Personnalisation insuffisante** : Bien que j'aie inclus des profils utilisateurs, l'approche par similarité cosinus peinait à capturer les relations complexes et non-linéaires entre utilisateurs, vidéos et schémas d'engagement.

5. **Approche binaire de l'engagement** : Mon premier modèle ne modélisait pas correctement la nature continue du ratio de visionnage, ce qui s'est avéré crucial étant donné sa distribution (0,05 à 135,66).

## Pourquoi l'Approche par Réseau Neuronal a Mieux Fonctionné

Mon modèle final de réseau neuronal a résolu ces limitations et montré des améliorations substantielles :

1. **Importance des caractéristiques basée sur les données** : Au lieu de pondérer manuellement les caractéristiques dans un score composite, le réseau neuronal a appris l'importance de chaque caractéristique directement à partir des données. Mon analyse de corrélation a montré que le taux d'achèvement était le signal positif le plus important (0,07), tandis que le modèle pouvait découvrir d'autres schémas subtils.

2. **Relations non-linéaires** : Les couches denses avec activations ReLU ont capturé des relations complexes non-linéaires entre les caractéristiques et le ratio de visionnage que la similarité cosinus ne pouvait pas modéliser.

3. **Personnalisation via embeddings** : Les embeddings de 32 dimensions pour les utilisateurs et les vidéos ont capturé des facteurs latents que la simple correspondance de contenu ne détectait pas. C'était particulièrement important compte tenu de la forte variance dans le comportement utilisateur (ratios de visionnage de 0,13 à 2,36).

4. **Régression vs similarité** : En prédisant directement le ratio de visionnage comme valeur continue plutôt qu'en calculant des scores de similarité, le modèle s'alignait mieux avec l'objectif réel de recommandation - trouver les vidéos avec le ratio de visionnage attendu le plus élevé.

5. **Meilleure approche d'évaluation** : Passer de la précision/rappel au Hit Ratio et NDCG a fourni des métriques qui reflétaient mieux la qualité du classement, ce qui est essentiel dans les systèmes de recommandation.

## Leçons Apprises

Cette expérience m'a enseigné plusieurs leçons précieuses sur les systèmes de recommandation :

1. **Commencer par l'analyse des données** : Mon approche finale a débuté par une EDA approfondie, révélant des insights que ma première tentative avait manqués, comme l'importance du taux d'achèvement et la corrélation négative du taux d'engagement.

2. **Plus simple peut être meilleur** : Malgré moins de caractéristiques ingéniées, le modèle de réseau neuronal a bien mieux performé en apprenant directement les bons schémas à partir des données.

3. **Choisir la bonne approche pour votre cible** : Aligner la sortie du modèle (prédiction du ratio de visionnage) avec l'objectif de recommandation (vidéos avec le ratio le plus élevé) a été crucial pour le succès.

4. **Les bonnes métriques sont importantes** : Utiliser Hit Ratio et NDCG a donné une image bien plus claire de la qualité des recommandations que les métriques standard de précision/rappel utilisées initialement.

L'amélioration de 0,048 de précision/rappel à 0,269 de Hit Ratio représente plus de 5 fois la performance initiale, avec l'avantage d'une excellente qualité de classement (0,719 NDCG). Cela démontre qu'une approche neuronale ciblée et basée sur les données surpasse même l'ingénierie de caractéristiques la plus soignée quand il s'agit de capturer la dynamique complexe des interactions utilisateur-vidéo.